# Land ice SMB model comparison
This notebook compares the downscaled output of surface mass balance (SMB) over the Greenland ice sheet (GrIS) to the regional model MAR. In what follows, we interchangeably call the MAR data "observation".
\
Note1: the MAR data are processed as a climatology spanning 1960-1999.\
Note2: the MAR data are available at a uniform resolution of 1km using the same projection as the CISM grid. This notebook requires the interpolation of the MAR data on the CISM grid. The interpolation is done in this notebook (for now) to allow for the eventuality of the CISM grid or the MAR grid to change in the future. \
creation: 05-26-24 \
contact: Gunter Leguy (gunterl@ucar.edu)

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as mcm
from netCDF4 import Dataset
import os
from scipy.interpolate import RegularGridInterpolator

# to display figures in notebook after executing the code.
%matplotlib inline

In [ ]:
# Parameter Defaults

CESM_output_dir = "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"
CESM2_output_dir = "/glade/campaign/cesm/collections/collections/CESM2-LE/timeseries/glc/proc/tseries/year_1/smb"

case_name = "b.e23_alpha17f.BLT1850.ne30_t232.092"  # case name
case_name2 = "f.e23_alpha17f.FLTHIST_ne30.roughtopo.099"  # case name
CESM2_case_name = "b.e21.BHISTsmbb.f09_g17.LE2-1251.019.cism.h.smb.1850-2015.nc"
path_obs = "/glade/u/home/gunterl/obs_diagnostic_cesm"  # path to observed dataset
obs_name = "GrIS_MARv3.12_climo_1960_1999.nc"
last_year = 2005

In [ ]:
path_glc = f"{CESM_output_dir}/{case_name}/glc/hist"  # path to glc output
file_glc = f"{path_glc}/{case_name}.cism.gris.initial_hist.0001-01-01-00000.nc"  # name of glc file output

path_cpl = f"{CESM_output_dir}/{case_name2}/cpl/hist"  # path to cpl output
file_cpl = f"{path_cpl}/{case_name2}.cpl.hx.1yr2glc.1996-01-01-00000.nc"  # name of last cpl simulation output

file_obs = f"{path_obs}/{obs_name}"  # name of observed dataset file
file_cesm2 = f"{CESM2_output_dir}/{CESM2_case_name}"  # name of CESM2 dataset

In [ ]:
## CISM grid information and loading a field used for filtering

# Reading the information we need from the glc file
nid = Dataset(file_glc, "r")
x_cism = nid.variables["x1"][:]
y_cism = nid.variables["y1"][:]
thk_cism = np.squeeze(nid.variables["thk"][0, :, :])
nid.close()

# Defining the grid dimensions
## For the CISM grid
nx_cism = len(x_cism)
ny_cism = len(y_cism)

In [ ]:
## The observed dataset
nid = Dataset(file_obs, "r")
x_obs = nid.variables["x"][:]
y_obs = nid.variables["y"][:]
smb_obs_src = np.squeeze(nid.variables["SMB"][0, :, :])
nid.close()

## For the observed grid
nx_obs = len(x_obs)
ny_obs = len(y_obs)

In [ ]:
# Constants
ny_climo = 10  # number of years to compute the climatology
res = np.abs(x_cism[1] - x_cism[0])  # CISM output resolution

rhoi = 917  # ice density kg/m3
rhow = 1000  # water density kg/m3
sec_in_yr = 60 * 60 * 24 * 365  # seconds in a year

smb_convert = sec_in_yr / rhoi * 1000  # converting kg m-2 s-1 ice to mm y-1 w.e.
kg_to_Gt = 1e-12  # Converting kg to Gt
mm_to_Gt = rhow * 1e-3 * res**2 * kg_to_Gt  # converting mm/yr to Gt/yr

In [ ]:
# Functions used in this notebook


def set_plot_prop_clean(ax):
    """
    This function cleans up the figures from unnecessary default figure properties.

    """
    ax.invert_yaxis()
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels("")
    ax.set_yticklabels("")
    ax.set_xticks([])
    ax.set_yticks([])


def rmse(prediction, target):
    """
    This function returns the root mean square error for the SMB.
    Input:
        prediction = field to predict
        target = field to compare with the prediction
    """
    return np.sqrt(((prediction - target) ** 2).mean())


def net_avrg(data):
    """
    This function returns the net average of a data field
    """
    return np.sum(np.sum(data, axis=0), axis=0)


def read_smb(file):
    """
    This function reads the CISM SMB dataset from a CESM simulation output
    in the cpl directory. The output is adjusted to be converted to mm/yr w.e unit.

    Input:
        file: name of the file to extract the SMB
    """
    nid = Dataset(file, "r")
    smb_cism = np.squeeze(nid.variables["glc1Exp_Flgl_qice"][0, :, :]) * smb_convert
    nid.close()
    return smb_cism

In [ ]:
# Loading the data
nid = Dataset(file_cpl, "r")
smb_cism = np.squeeze(nid.variables["glc1Exp_Flgl_qice"][0, :, :]) * smb_convert
nid.close()

In [ ]:
# creating the SMB climatology for comparison with observation

# Initializing a field for the climatology
smb_cism_climo = np.zeros((ny_cism, nx_cism))

# Counter for available year (only needed if the number of years available is smaller
# than the number of years requested to create the climatology.
count_yr = 0

for k in range(ny_climo):

    year_to_read = last_year - k
    str_yr = str(year_to_read).zfill(4)
    file_cpl = f"{path_cpl}/{case_name2}.cpl.hx.1yr2glc.{str_yr}-01-01-00000.nc"

    if not os.path.isfile(file_cpl):
        print("The couple file for time", str_yr, "does not exist.")
        print(
            "We will only use the files that existed until now to create the SMB climatology."
        )
        break

    smb_cism_climo = smb_cism_climo + read_smb(file_cpl)
    count_yr = count_yr + 1

# Averaging the climo data
smb_cism_climo = smb_cism_climo / count_yr

print("number of years used in climatology = ", count_yr)

In [ ]:
# Interpolating the observed data onto the CISM grid

# Defining the interpolation functions
myInterpFunction_smb_obs = RegularGridInterpolator(
    (x_obs, y_obs),
    smb_obs_src.transpose(),
    method="linear",
    bounds_error=False,
    fill_value=None,
)

# Initializing the glacier ID variable
smb_obs_cism = np.zeros((ny_cism, nx_cism))

# Performing the interpolation
for j in range(ny_cism):
    point_y = np.zeros(nx_cism)
    point_y[:] = y_cism[j]
    pts = (x_cism[:], point_y[:])
    smb_obs_cism[j, :] = myInterpFunction_smb_obs(pts)

In [ ]:
# Filtering out fill values
smb_obs_cism = np.where(smb_obs_cism > 1e20, 0, smb_obs_cism)
mask = thk_cism[:, :] == 0
smb_obs_cism = np.where(mask, 0, smb_obs_cism)
smb_cism = np.where(mask, 0, smb_cism)
smb_cism_climo = np.where(mask, 0, smb_cism_climo)

In [ ]:
# Comparing the SMB

# Computing the SMB difference
smb_diff_cesm3 = smb_cism_climo - smb_obs_cism

# Computing the net averages SMB
avg_smb_cism_climo = np.round(net_avrg(smb_cism_climo) * mm_to_Gt, 2)
avg_smb_obs_cism = np.round(net_avrg(smb_obs_cism) * mm_to_Gt, 2)
avg_smb_diff_cesm3 = np.round(net_avrg(smb_diff_cesm3) * mm_to_Gt, 2)

# Colormap choice
my_cmap = mcm.get_cmap("Spectral")
# my_cmap = mcm.get_cmap('RdYlBu_r')
my_cmap_diff = mcm.get_cmap("bwr_r")


# Colorbar bounds
vmin = -2000
vmax = 2000

# Figure
fig, ax = plt.subplots(1, 3, sharey=True, figsize=[22, 9])

## Left panel
last_panel0 = ax[0].imshow(smb_cism_climo[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)
ax[0].set_title("SMB CESM3 (mm/y w.e.)", fontsize=16)
set_plot_prop_clean(ax[0])
ax[0].annotate("net avg =" + str(avg_smb_cism_climo) + " Gt/yr", xy=(5, 5), fontsize=16)

pos = ax[0].get_position()
cax = fig.add_axes([0.35, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel0, cax=cax)
cbar.ax.tick_params(labelsize=16)


## Center panel
last_panel1 = ax[1].imshow(smb_obs_cism[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)

ax[1].set_title("SMB Obs (mm/y w.e.)", fontsize=16)
set_plot_prop_clean(ax[1])
ax[1].annotate("net avg =" + str(avg_smb_obs_cism) + " Gt/yr", xy=(5, 5), fontsize=16)


pos = ax[1].get_position()
cax = fig.add_axes([0.62, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel1, cax=cax)
cbar.ax.tick_params(labelsize=16)


## Right panel

vmin = -2000
vmax = 2000

last_panel2 = ax[2].imshow(smb_diff_cesm3, vmin=vmin, vmax=vmax, cmap=my_cmap_diff)

ax[2].set_title("SMB diff (CESM3 - Obs, mm/yr w.e.)", fontsize=16)
set_plot_prop_clean(ax[2])

ax[2].annotate("net avg =" + str(avg_smb_diff_cesm3) + " Gt/yr", xy=(5, 5), fontsize=16)

pos = ax[2].get_position()
cax = fig.add_axes([0.89, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel2, cax=cax)
cbar.ax.tick_params(labelsize=16)

fig.subplots_adjust(right=0.89)

In [ ]:
# Integrated SMB time series

# Initializing a field for the climatology
avg_smb_cism_timeseries = np.zeros(last_year)

# Counter for available year (only needed if the number of years available is smaller
# than the number of years requested to create the climatology.
count_yr = 0

for k in range(last_year):

    year_to_read = last_year - k
    str_yr = str(year_to_read).zfill(4)
    file_cpl = f"{path_cpl}/{case_name2}.cpl.hx.1yr2glc." + str_yr + "-01-01-00000.nc"

    if not os.path.isfile(file_cpl):
        print("The couple file for time", str_yr, "does not exist.")
        print(
            "We will only use the files that existed until now to create the time series."
        )
        break

    smb_temp = read_smb(file_cpl)
    smb_temp = np.where(mask, 0, smb_temp)

    avg_smb_cism_timeseries[year_to_read - 1] = np.round(
        net_avrg(smb_temp) * mm_to_Gt, 2
    )
    count_yr = count_yr + 1

    del smb_temp

first_year = year_to_read

print("number of years used in climatology = ", count_yr)

In [ ]:
# Plotting the SMB spatially averaged time series


time = np.arange(first_year, last_year)
nt = len(time)


avg_smb_climo_timeseries = np.zeros(nt)
avg_smb_obs_cism_timeseries = np.zeros(nt)

avg_smb_climo_timeseries[:] = avg_smb_cism_climo
avg_smb_obs_cism_timeseries[:] = avg_smb_obs_cism


x_ticks = np.arange(first_year, last_year, 10)
tickx = x_ticks

ymin = 150
ymax = 650
y_step = 50
y_ticks = np.arange(ymin, ymax + y_step, y_step)


plt.figure(figsize=(16, 7))

line = "-"
color = "blue"
sizefont = 16
linewidth = 2


# Plotting NESM3 experiments
plt.subplot(111)

label = "CISM"
plt.plot(
    time,
    avg_smb_cism_timeseries[first_year::],
    line,
    ms=3,
    mfc=color,
    color=color,
    label=label,
    linewidth=linewidth,
)

label = "CISM climatology"
color = "k"
plt.plot(
    time,
    avg_smb_climo_timeseries[:],
    line,
    ms=3,
    mfc=color,
    color=color,
    label=label,
    linewidth=linewidth,
)

label = "Observation"
color = "r"
plt.plot(
    time,
    avg_smb_obs_cism_timeseries[:],
    line,
    ms=3,
    mfc=color,
    color=color,
    label=label,
    linewidth=linewidth,
)


plt.xlim([first_year, last_year])
plt.xticks(x_ticks, tickx, fontsize=sizefont)
plt.xlabel(r"$Time$ (y)", fontsize=sizefont)
plt.ylabel("SMB average evolution (Gt/yr)", multialignment="center", fontsize=sizefont)
plt.ylim([ymin, ymax])
plt.yticks(fontsize=sizefont)
plt.legend(loc="upper left", ncol=1, frameon=True, borderaxespad=0)

plt.title("SMB average evolution", fontsize=sizefont)

# Adding comparison with CESM2

In [ ]:
# Defining the paths and files for CESM2


nid = Dataset(file_cesm2, "r")
smb_cesm2 = np.squeeze(nid.variables["smb"][0, :, :])
nid.close()

In [ ]:
smb_cesm2 = np.where(mask, 0, smb_cesm2)

In [ ]:
# Comparing the SMB

# Computing the SMB difference
# smb_diff = smb_cism_climo - smb_obs_cism

# Computing the net averages SMB
avg_smb_cesm2 = np.round(net_avrg(smb_cesm2) * mm_to_Gt, 2)


# Colormap choice
my_cmap = mcm.get_cmap("Spectral")
# my_cmap = mcm.get_cmap('RdYlBu_r')
my_cmap_diff = mcm.get_cmap("bwr_r")


# Colorbar bounds
vmin = -2000
vmax = 2000

# Figure
fig, ax = plt.subplots(1, 3, sharey=True, figsize=[22, 9])

## Left panel
last_panel0 = ax[0].imshow(smb_cesm2[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)
ax[0].set_title("SMB CESM2 (mm/y w.e.)", fontsize=16)
set_plot_prop_clean(ax[0])
ax[0].annotate("net avg =" + str(avg_smb_cesm2) + " Gt/yr", xy=(5, 5), fontsize=16)

pos = ax[0].get_position()
cax = fig.add_axes([0.35, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel0, cax=cax)
cbar.ax.tick_params(labelsize=16)


## Center panel
last_panel1 = ax[1].imshow(smb_cism_climo[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)

ax[1].set_title("SMB CESM3 (mm/y w.e.)", fontsize=16)
set_plot_prop_clean(ax[1])
ax[1].annotate("net avg =" + str(avg_smb_cism_climo) + " Gt/yr", xy=(5, 5), fontsize=16)


pos = ax[1].get_position()
cax = fig.add_axes([0.62, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel1, cax=cax)
cbar.ax.tick_params(labelsize=16)


## Right panel

last_panel2 = ax[2].imshow(smb_obs_cism[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)

ax[2].set_title("SMB Obs (mm/y w.e.)", fontsize=16)
set_plot_prop_clean(ax[2])
ax[2].annotate("net avg =" + str(avg_smb_obs_cism) + " Gt/yr", xy=(5, 5), fontsize=16)


pos = ax[2].get_position()
cax = fig.add_axes([0.62, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel1, cax=cax)
cbar.ax.tick_params(labelsize=16)


# last_panel2 = ax[2].imshow(smb_diff_climo, vmin=vmin, vmax=vmax, cmap=my_cmap)

# ax[2].set_title('SMB diff (CISM - Obs, mm/yr w.e.)',fontsize=16)
# set_plot_prop_clean(ax[2])

# ax[2].annotate('net avg =' + str(avg_smb_diff) + ' Gt/yr',xy=(5,5),fontsize=16)

# pos = ax[2].get_position()
# cax = fig.add_axes([0.89, pos.y0, 0.02, pos.y1 - pos.y0])

# cbar = fig.colorbar(last_panel2,cax=cax)
# cbar.ax.tick_params(labelsize=16)

fig.subplots_adjust(right=0.89)

In [ ]:
# Computing the difference w.r.t MAR obs
smb_diff_cesm2 = smb_cesm2 - smb_obs_cism

# Computing the net averages SMB
avg_smb_cesm2 = np.round(net_avrg(smb_cesm2) * mm_to_Gt, 2)
avg_smb_diff_cesm2 = np.round(net_avrg(smb_diff_cesm2) * mm_to_Gt, 2)

# Masking out ocean cells
mask2 = smb_cesm2[:, :] == 0
smb_cesm2_plot = np.where(mask2, np.float64("Nan"), smb_cesm2)
smb_obs_cism_plot = np.where(mask2, np.float64("Nan"), smb_obs_cism)
smb_cism_climo_plot = np.where(mask2, np.float64("Nan"), smb_cism_climo)

In [ ]:
# Comparing the SMB

# Computing the SMB difference
# smb_diff = smb_cism_climo - smb_obs_cism


# Colormap choice
my_cmap = mcm.get_cmap("Spectral")
# my_cmap = mcm.get_cmap('RdYlBu_r')
my_cmap_diff = mcm.get_cmap("bwr_r")


# Colorbar bounds
vmin = -2000
vmax = 2000

# Figure
fig, ax = plt.subplots(1, 3, sharey=True, figsize=[16, 9])

## Left panel
last_panel0 = ax[0].imshow(smb_cesm2_plot[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)
# ax[0].set_title('SMB CESM2 (mm/y w.e.)',fontsize=16)
ax[0].set_title("CESM2", fontsize=16)
set_plot_prop_clean(ax[0])
ax[0].annotate("net avg =" + str(avg_smb_cesm2) + " Gt/yr", xy=(5, 5), fontsize=16)
ax[0].set_ylabel("SMB (mm/y w.e)", fontsize=16)
# pos = ax[0].get_position()
# cax = fig.add_axes([0.35, pos.y0, 0.02, pos.y1 - pos.y0])

# cbar = fig.colorbar(last_panel0, cax=cax)
# cbar.ax.tick_params(labelsize=16)


## Center panel
last_panel1 = ax[1].imshow(
    smb_cism_climo_plot[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap
)

ax[1].set_title("CESM3", fontsize=16)
set_plot_prop_clean(ax[1])
ax[1].annotate("net avg =" + str(avg_smb_cism_climo) + " Gt/yr", xy=(5, 5), fontsize=16)


# pos = ax[1].get_position()
# cax = fig.add_axes([0.62, pos.y0, 0.02, pos.y1 - pos.y0])

# cbar = fig.colorbar(last_panel1,cax=cax)
# cbar.ax.tick_params(labelsize=16)


## Right panel

last_panel2 = ax[2].imshow(smb_obs_cism_plot[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap)

ax[2].set_title("MARv3.12 1960-1999", fontsize=16)
set_plot_prop_clean(ax[2])
ax[2].annotate("net avg =" + str(avg_smb_obs_cism) + " Gt/yr", xy=(5, 5), fontsize=16)


pos = ax[2].get_position()
cax = fig.add_axes([0.9, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel1, cax=cax)
cbar.ax.tick_params(labelsize=16)


# last_panel2 = ax[2].imshow(smb_diff_climo, vmin=vmin, vmax=vmax, cmap=my_cmap)

# ax[2].set_title('SMB diff (CISM - Obs, mm/yr w.e.)',fontsize=16)
# set_plot_prop_clean(ax[2])

# ax[2].annotate('net avg =' + str(avg_smb_diff) + ' Gt/yr',xy=(5,5),fontsize=16)

# pos = ax[2].get_position()
# cax = fig.add_axes([0.89, pos.y0, 0.02, pos.y1 - pos.y0])

# cbar = fig.colorbar(last_panel2,cax=cax)
# cbar.ax.tick_params(labelsize=16)

fig.subplots_adjust(right=0.89)

In [ ]:
# Comparing the SMB

# Colormap choice
my_cmap = mcm.get_cmap("Spectral")
# my_cmap = mcm.get_cmap('RdYlBu_r')
my_cmap_diff = mcm.get_cmap("bwr_r")


# Colorbar bounds
vmin = -2000
vmax = 2000

# Figure
fig, ax = plt.subplots(1, 2, sharey=True, figsize=[12, 9])

## Left panel
last_panel0 = ax[0].imshow(
    smb_diff_cesm2[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap_diff
)
ax[0].set_title("CESM2 - MAR", fontsize=16)
set_plot_prop_clean(ax[0])
ax[0].annotate("net avg =" + str(avg_smb_diff_cesm2) + " Gt/yr", xy=(5, 5), fontsize=16)
ax[0].set_ylabel("SMB difference (mm/yr w.e.)", fontsize=16)

# pos = ax[0].get_position()
# cax = fig.add_axes([0.465, pos.y0, 0.02, pos.y1 - pos.y0])

# cbar = fig.colorbar(last_panel0, cax=cax)
# cbar.ax.tick_params(labelsize=16)


## Right panel
last_panel1 = ax[1].imshow(
    smb_diff_cesm3[:, :], vmin=vmin, vmax=vmax, cmap=my_cmap_diff
)

ax[1].set_title("CESM3 - MAR", fontsize=16)
set_plot_prop_clean(ax[1])
ax[1].annotate("net avg =" + str(avg_smb_diff_cesm3) + " Gt/yr", xy=(5, 5), fontsize=16)

pos = ax[1].get_position()
cax = fig.add_axes([0.89, pos.y0, 0.02, pos.y1 - pos.y0])

cbar = fig.colorbar(last_panel1, cax=cax)
cbar.ax.tick_params(labelsize=16)


# fig.tight_layout()
fig.subplots_adjust(right=0.89)